# Методы оценки важности признаков

В этом ноутбуке рассмотрим различные методы оценки важности признаков:
1. Веса линейных моделей (коэффициенты)
2. Feature importance из деревьев (Gini/Entropy importance)
3. Permutation importance
4. SHAP (Shapley Additive Explanations)
5. LIME (Local Interpretable Model-agnostic Explanations)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

# Установите если нужно: pip install shap
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("SHAP не установлен. Установите: pip install shap")

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Подготовка данных

In [ ]:
# Загрузим датасет breast cancer
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

print(f"Размер данных: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")
print(f"\nПервые признаки:\n{X.columns[:5].tolist()}")

# Разделим на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Нормализация для линейных моделей
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 1. Веса линейных моделей

Коэффициенты линейных моделей показывают вклад каждого признака в предсказание.

In [ ]:
# Логистическая регрессия
lr = LogisticRegression(random_state=42, max_iter=10000)
lr.fit(X_train_scaled, y_train)

# Получаем коэффициенты
coef_importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': lr.coef_[0],
    'abs_coefficient': np.abs(lr.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print(f"Accuracy на тесте: {lr.score(X_test_scaled, y_test):.3f}\n")
print("Топ-10 признаков по абсолютному значению коэффициентов:")
print(coef_importance.head(10))

In [ ]:
# Визуализация
plt.figure(figsize=(10, 6))
top_n = 15
top_features = coef_importance.head(top_n)
colors = ['red' if x < 0 else 'blue' for x in top_features['coefficient']]
plt.barh(range(top_n), top_features['coefficient'], color=colors)
plt.yticks(range(top_n), top_features['feature'])
plt.xlabel('Coefficient Value')
plt.title('Top 15 Features by Logistic Regression Coefficients')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 2. Feature Importance из деревьев

Древесные модели вычисляют важность на основе уменьшения Gini impurity или энтропии.

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(f"Random Forest Accuracy: {rf.score(X_test, y_test):.3f}\n")
print("Топ-10 признаков:")
print(rf_importance.head(10))

In [ ]:
# Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)

gb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': gb.feature_importances_
}).sort_values('importance', ascending=False)

print(f"Gradient Boosting Accuracy: {gb.score(X_test, y_test):.3f}\n")
print("Топ-10 признаков:")
print(gb_importance.head(10))

In [ ]:
# Сравнение важности между RF и GB
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

top_n = 15
axes[0].barh(range(top_n), rf_importance.head(top_n)['importance'])
axes[0].set_yticks(range(top_n))
axes[0].set_yticklabels(rf_importance.head(top_n)['feature'])
axes[0].set_xlabel('Importance')
axes[0].set_title('Random Forest Feature Importance')
axes[0].invert_yaxis()

axes[1].barh(range(top_n), gb_importance.head(top_n)['importance'])
axes[1].set_yticks(range(top_n))
axes[1].set_yticklabels(gb_importance.head(top_n)['feature'])
axes[1].set_xlabel('Importance')
axes[1].set_title('Gradient Boosting Feature Importance')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 3. Permutation Importance

Измеряет уменьшение качества модели при случайной перестановке значений признака.

In [ ]:
# Permutation importance для Random Forest
perm_importance = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)

perm_imp_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("Топ-10 признаков по Permutation Importance:")
print(perm_imp_df.head(10))

In [ ]:
# Визуализация с доверительными интервалами
plt.figure(figsize=(10, 6))
top_n = 15
top_perm = perm_imp_df.head(top_n)
y_pos = range(top_n)

plt.barh(y_pos, top_perm['importance_mean'])
plt.errorbar(top_perm['importance_mean'], y_pos, 
             xerr=top_perm['importance_std'], fmt='none', 
             color='black', capsize=3)
plt.yticks(y_pos, top_perm['feature'])
plt.xlabel('Permutation Importance')
plt.title('Top 15 Features by Permutation Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 4. SHAP (Shapley Additive Explanations)

Основан на теории игр Шепли - показывает вклад каждого признака в предсказание.

In [ ]:
if SHAP_AVAILABLE:
    # TreeExplainer для Random Forest
    explainer = shap.TreeExplainer(rf)
    shap_values = explainer.shap_values(X_test)
    
    # Для бинарной классификации берем значения для положительного класса
    if isinstance(shap_values, list):
        shap_values_class = shap_values[1]
    else:
        shap_values_class = shap_values
    
    # Средняя абсолютная важность
    shap_importance = pd.DataFrame({
        'feature': X.columns,
        'shap_importance': np.abs(shap_values_class).mean(axis=0)
    }).sort_values('shap_importance', ascending=False)
    
    print("Топ-10 признаков по SHAP values:")
    print(shap_importance.head(10))
else:
    print("SHAP не установлен. Пропускаем этот раздел.")

In [ ]:
if SHAP_AVAILABLE:
    # Summary plot
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values_class, X_test, plot_type="bar", show=False)
    plt.tight_layout()
    plt.show()

In [ ]:
if SHAP_AVAILABLE:
    # Детальный summary plot с распределением значений
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values_class, X_test, show=False)
    plt.tight_layout()
    plt.show()

In [ ]:
if SHAP_AVAILABLE:
    # Dependence plot для топового признака
    top_feature = shap_importance.iloc[0]['feature']
    plt.figure(figsize=(10, 6))
    shap.dependence_plot(
        top_feature, shap_values_class, X_test, show=False
    )
    plt.tight_layout()
    plt.show()

## 5. Сравнение всех методов

Сравним рейтинги признаков по разным методам.

In [ ]:
# Создадим сводную таблицу
comparison = pd.DataFrame({'feature': X.columns})

# Добавим ранги по каждому методу
comparison = comparison.merge(
    coef_importance[['feature', 'abs_coefficient']].rename(
        columns={'abs_coefficient': 'logreg_coef'}
    ), on='feature'
)
comparison = comparison.merge(
    rf_importance[['feature', 'importance']].rename(
        columns={'importance': 'rf_importance'}
    ), on='feature'
)
comparison = comparison.merge(
    gb_importance[['feature', 'importance']].rename(
        columns={'importance': 'gb_importance'}
    ), on='feature'
)
comparison = comparison.merge(
    perm_imp_df[['feature', 'importance_mean']].rename(
        columns={'importance_mean': 'perm_importance'}
    ), on='feature'
)

if SHAP_AVAILABLE:
    comparison = comparison.merge(
        shap_importance[['feature', 'shap_importance']], on='feature'
    )

# Вычислим ранги
for col in comparison.columns[1:]:
    comparison[f'{col}_rank'] = comparison[col].rank(ascending=False)

print("Сравнение методов (топ-15):")
rank_cols = [col for col in comparison.columns if col.endswith('_rank')]
comparison['mean_rank'] = comparison[rank_cols].mean(axis=1)
comparison_sorted = comparison.sort_values('mean_rank')
print(comparison_sorted[['feature'] + rank_cols + ['mean_rank']].head(15))

In [ ]:
# Тепловая карта рангов
top_features = comparison_sorted.head(20)['feature'].tolist()
rank_data = comparison[comparison['feature'].isin(top_features)]
rank_data = rank_data.set_index('feature')[rank_cols]

plt.figure(figsize=(10, 10))
sns.heatmap(rank_data, annot=True, fmt='.0f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Rank'})
plt.title('Feature Importance Ranks Across Methods\n(Lower is better)')
plt.xlabel('Method')
plt.ylabel('Feature')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Выводы

### Плюсы и минусы методов:

**1. Веса линейных моделей:**
- ✅ Быстро, интерпретируемо
- ❌ Только для линейных моделей, требует нормализации

**2. Tree-based importance:**
- ✅ Быстро, работает "из коробки"
- ❌ Смещение к признакам с большим количеством категорий/значений

**3. Permutation importance:**
- ✅ Model-agnostic, учитывает взаимодействия
- ❌ Медленнее, может быть нестабильным

**4. SHAP:**
- ✅ Теоретически обоснован, показывает направление влияния
- ❌ Вычислительно затратен, сложнее интерпретация

**Рекомендации:**
- Используйте несколько методов для надежности
- SHAP - золотой стандарт, но требует времени
- Permutation importance - хороший компромисс
- Tree importance - быстрая оценка